## SVM Implementation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearnex.svm import SVC
from sklearn import metrics
from sklearn.model_selection import cross_val_score,GridSearchCV,train_test_split
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,StandardScaler

In [2]:
# Dataset> you can get the dataset details here
# https://www.kaggle.com/ruslankl/mice-protein-expression
# https://archive.ics.uci.edu/ml/machine-learning-databases/00342/

In [3]:
#https://www.kaggle.com/emmaweizhang/knn-and-decision-tree-classifiers

In [4]:
df_mice = pd.read_csv("https://raw.githubusercontent.com/atulpatelDS/Data_Files/master/Mice_Protein/Data_Cortex_Nuclear.csv")

In [5]:
df_mice.head()

,MouseID,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,pAKT_N,pBRAF_N,pCAMKII_N,pCREB_N,...,pCFOS_N,SYP_N,H3AcK18_N,EGR1_N,H3MeK4_N,CaNA_N,Genotype,Treatment,Behavior,class
0,309_1,0.503644,0.747193,0.430175,2.816329,5.990152,0.218830,0.177565,2.373744,0.232224,...,0.108336,0.427099,0.114783,0.131790,0.128186,1.675652,Control,Memantine,C/S,c-CS-m
1,309_2,0.514617,0.689064,0.411770,2.789514,5.685038,0.211636,0.172817,2.292150,0.226972,...,0.104315,0.441581,0.111974,0.135103,0.131119,1.743610,Control,Memantine,C/S,c-CS-m
2,309_3,0.509183,0.730247,0.418309,2.687201,5.622059,0.209011,0.175722,2.283337,0.230247,...,0.106219,0.435777,0.111883,0.133362,0.127431,1.926427,Control,Memantine,C/S,c-CS-m
3,309_4,0.442107,0.617076,0.358626,2.466947,4.979503,0.222886,0.176463,2.152301,0.207004,...,0.111262,0.391691,0.130405,0.147444,0.146901,1.700563,Control,Memantine,C/S,c-CS-m
4,309_5,0.434940,0.617430,0.358802,2.365785,4.718679,0.213106,0.173627,2.134014,0.192158,...,0.110694,0.434154,0.118481,0.140314,0.148380,1.839730,Control,Memantine,C/S,c-CS-m


In [6]:
df_mice.isnull().any()

MouseID      False
DYRK1A_N      True
ITSN1_N       True
BDNF_N        True
NR1_N         True
             ...  
CaNA_N       False
Genotype     False
Treatment    False
Behavior     False
class        False
Length: 82, dtype: bool

In [7]:
# Check the null values in the columns
columns_null_value = df_mice.columns[df_mice.isnull().any()==True].tolist()

In [8]:
# Check the percentage of missing values in columns
for col in columns_null_value:
    print("Column {} missing value : {}".format(col,df_mice[col].isnull().sum()*100/df_mice.shape[0]))

Column DYRK1A_N missing value : 0.2777777777777778
Column ITSN1_N missing value : 0.2777777777777778
Column BDNF_N missing value : 0.2777777777777778
Column NR1_N missing value : 0.2777777777777778
Column NR2A_N missing value : 0.2777777777777778
Column pAKT_N missing value : 0.2777777777777778
Column pBRAF_N missing value : 0.2777777777777778
Column pCAMKII_N missing value : 0.2777777777777778
Column pCREB_N missing value : 0.2777777777777778
Column pELK_N missing value : 0.2777777777777778
Column pERK_N missing value : 0.2777777777777778
Column pJNK_N missing value : 0.2777777777777778
Column PKCA_N missing value : 0.2777777777777778
Column pMEK_N missing value : 0.2777777777777778
Column pNR1_N missing value : 0.2777777777777778
Column pNR2A_N missing value : 0.2777777777777778
Column pNR2B_N missing value : 0.2777777777777778
Column pPKCAB_N missing value : 0.2777777777777778
Column pRSK_N missing value : 0.2777777777777778
Column AKT_N missing value : 0.2777777777777778
Column BRA

In [9]:
# Check the percentage of missing values columns
cols_missing_less_1_per = []
cols_missing_greater_1_per = []
for col in columns_null_value:
    if df_mice[col].isnull().sum()*100/df_mice.shape[0] < 1:
        cols_missing_less_1_per.append(col)
    else:
        cols_missing_greater_1_per.append(col)       

In [10]:
cols_missing_less_1_per

['DYRK1A_N',
 'ITSN1_N',
 'BDNF_N',
 'NR1_N',
 'NR2A_N',
 'pAKT_N',
 'pBRAF_N',
 'pCAMKII_N',
 'pCREB_N',
 'pELK_N',
 'pERK_N',
 'pJNK_N',
 'PKCA_N',
 'pMEK_N',
 'pNR1_N',
 'pNR2A_N',
 'pNR2B_N',
 'pPKCAB_N',
 'pRSK_N',
 'AKT_N',
 'BRAF_N',
 'CAMKII_N',
 'CREB_N',
 'ERK_N',
 'GSK3B_N',
 'JNK_N',
 'MEK_N',
 'TRKA_N',
 'RSK_N',
 'APP_N',
 'SOD1_N',
 'MTOR_N',
 'P38_N',
 'pMTOR_N',
 'DSCR1_N',
 'AMPKA_N',
 'NR2B_N',
 'pNUMB_N',
 'RAPTOR_N',
 'TIAM1_N',
 'pP70S6_N']

In [11]:
cols_missing_greater_1_per

['ELK_N',
 'Bcatenin_N',
 'BAD_N',
 'BCL2_N',
 'pCFOS_N',
 'H3AcK18_N',
 'EGR1_N',
 'H3MeK4_N']

In [12]:
# remove missing value from those columns which has percentage less than 1%
for col in cols_missing_less_1_per:
    df_mice.dropna(axis=0,subset=[col],inplace=True)

In [13]:
# Replace null values with mean of those columns where missing value is greater than 1%
for col in cols_missing_greater_1_per:
    df_mice[col].fillna(df_mice[col].mean(),inplace=True)

In [14]:
df_mice.isnull().sum()

MouseID      0
DYRK1A_N     0
ITSN1_N      0
BDNF_N       0
NR1_N        0
            ..
CaNA_N       0
Genotype     0
Treatment    0
Behavior     0
class        0
Length: 82, dtype: int64

In [15]:
df_mice.MouseID.value_counts()

309_1       1
3411_6      1
293_7       1
293_8       1
293_9       1
           ..
3484_8      1
3484_9      1
3484_10     1
3484_11     1
J3295_15    1
Name: MouseID, Length: 1073, dtype: int64

In [16]:
# remove the mouse_id column as this has unique value in each rows
df_mice.drop(columns=["MouseID"],axis=1,inplace=True)

In [17]:
columns_string = df_mice.select_dtypes(include="object").columns.to_list()

In [18]:
columns_string

['Genotype', 'Treatment', 'Behavior', 'class']

In [19]:
for col in columns_string:    
    print("Unique value in column {} :\n {} \n".format(col,df_mice[col].value_counts()))

Unique value in column Genotype :
 Control    566
Ts65Dn     507
Name: Genotype, dtype: int64 

Unique value in column Treatment :
 Memantine    570
Saline       503
Name: Treatment, dtype: int64 

Unique value in column Behavior :
 S/C    552
C/S    521
Name: Behavior, dtype: int64 

Unique value in column class :
 c-CS-m    150
c-SC-m    150
c-SC-s    135
t-CS-m    135
t-SC-m    135
t-SC-s    132
c-CS-s    131
t-CS-s    105
Name: class, dtype: int64 



According to the dataset description : https://www.kaggle.com/ruslankl/mice-protein-expression, column "class" contain the information in Genotype, Treatment and Behavior so we can drop all these 3 columns.

In [20]:
df_mice.drop(columns=["Genotype","Treatment","Behavior"],axis=1,inplace=True)

In [21]:
column_string = df_mice.select_dtypes(include="object").columns.to_list()
column_string

['class']

In [22]:
# Lets convert the categorical column "Class" into numerical

In [23]:
lab_enc = LabelEncoder()

In [24]:
for col in column_string:
    df_mice[col] = lab_enc.fit_transform(df_mice[col])

In [25]:
df_mice.head()

,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,pAKT_N,pBRAF_N,pCAMKII_N,pCREB_N,pELK_N,...,BAD_N,BCL2_N,pS6_N,pCFOS_N,SYP_N,H3AcK18_N,EGR1_N,H3MeK4_N,CaNA_N,class
0,0.503644,0.747193,0.430175,2.816329,5.990152,0.218830,0.177565,2.373744,0.232224,1.750936,...,0.122652,0.134757,0.106305,0.108336,0.427099,0.114783,0.131790,0.128186,1.675652,0
1,0.514617,0.689064,0.411770,2.789514,5.685038,0.211636,0.172817,2.292150,0.226972,1.596377,...,0.116682,0.134757,0.106592,0.104315,0.441581,0.111974,0.135103,0.131119,1.743610,0
2,0.509183,0.730247,0.418309,2.687201,5.622059,0.209011,0.175722,2.283337,0.230247,1.561316,...,0.118508,0.134757,0.108303,0.106219,0.435777,0.111883,0.133362,0.127431,1.926427,0
3,0.442107,0.617076,0.358626,2.466947,4.979503,0.222886,0.176463,2.152301,0.207004,1.595086,...,0.132781,0.134757,0.103184,0.111262,0.391691,0.130405,0.147444,0.146901,1.700563,0
4,0.434940,0.617430,0.358802,2.365785,4.718679,0.213106,0.173627,2.134014,0.192158,1.504230,...,0.129954,0.134757,0.104784,0.110694,0.434154,0.118481,0.140314,0.148380,1.839730,0


In [26]:
for col in column_string:    
    print("Unique value in column {} :\n {} \n".format(col,df_mice[col].value_counts()))

Unique value in column class :
 0    150
2    150
3    135
4    135
6    135
7    132
1    131
5    105
Name: class, dtype: int64 



In [27]:
# We can also check the outliers and some other feature engineering tasks.
# we can aslo check the feature selection
# these things I am not covering here.you can refer my Feature Engineering videos

In [28]:
print(df_mice.shape)
print(type(df_mice))

(1073, 78)
<class 'pandas.core.frame.DataFrame'>


In [29]:
X = df_mice.drop(columns=["class"],axis=1).values
y = df_mice["class"].values

In [30]:
print(X.shape)
print(type(X))
print(y.shape)
print(type(y))

(1073, 77)
<class 'numpy.ndarray'>
(1073,)
<class 'numpy.ndarray'>


In [31]:
# Display the correlated features
def correlation(dataset, threshold):
    col_corr = [] # those cols which have coeff greater than threshold value
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j] >= threshold) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.append(colname)
    return col_corr                

In [32]:
# Display those columsn which have correlation more than 90%
corr_columns = correlation(df_mice.drop(columns=["class"],axis=1),0.9)
corr_columns

['ITSN1_N', 'pERK_N', 'pNR1_N', 'BRAF_N', 'Bcatenin_N', 'pS6_N']

In [33]:
df_mice.drop(columns=corr_columns,axis=1,inplace=True)

In [34]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [35]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(751, 77)
(322, 77)
(751,)
(322,)


In [36]:
#Rescale the training and testing datasetDataset

In [37]:
scale = MinMaxScaler()

In [38]:
scale.fit(X_train)

MinMaxScaler()

In [39]:
X_train_scale = scale.transform(X_train)

In [40]:
X_test_scale = scale.transform(X_test)

In [41]:
# Apply the SVM Classifier

In [42]:
kernel = ["poly", "rbf", "sigmoid","linear"]
for ker in kernel:
    svm_cf = SVC(
    C=1.0,
    kernel=ker,
    degree=3,
    gamma='scale',
    coef0=0.0,
    shrinking=True,
    probability=False,
    tol=0.001,
    cache_size=200,
    class_weight=None,
    verbose=False,
    max_iter=-1,
    decision_function_shape='ovo',
    break_ties=False,
    random_state=None)
    svm_cf.fit(X_train_scale,y_train)
    y_pred = svm_cf.predict(X_test_scale)
    print("Model accuracy with kernel {} is {}".format(ker,metrics.accuracy_score(y_test,y_pred)))
    

Model accuracy with kernel poly is 0.9937888198757764
Model accuracy with kernel rbf is 0.9968944099378882
Model accuracy with kernel sigmoid is 0.13975155279503104
Model accuracy with kernel linear is 0.9782608695652174


In [43]:
# get support vectors
kernel = ["poly", "rbf", "sigmoid","linear"]
for ker in kernel:
    svm_cf = SVC(
    C=1.0,
    kernel=ker,
    degree=3,
    gamma='scale',
    coef0=0.0,
    shrinking=True,
    probability=False,
    tol=0.001,
    cache_size=200,
    class_weight=None,
    verbose=False,
    max_iter=-1,
    decision_function_shape='ovo',
    break_ties=False,
    random_state=None)
    svm_cf.fit(X_train_scale,y_train)
    y_pred = svm_cf.predict(X_test_scale)
    print("Support Vectors with indices of kernel",ker," are : \n \n",list(zip(svm_cf.support_,svm_cf.support_vectors_)))
    

Support Vectors with indices of kernel poly  are : 
 
 [(0, array([0.20367512, 0.25270237, 0.74711033, 0.67468157, 0.53569653,
       0.27046498, 0.35010265, 0.42388596, 0.53040324, 0.23093362,
       0.24603847, 0.60285946, 0.73081107, 0.36346333, 0.65211801,
       0.71832378, 0.66643732, 0.58560325, 0.59915617, 0.55007033,
       0.19568069, 0.37260778, 0.27130791, 0.62035233, 0.56618506,
       0.51817584, 0.58081112, 0.79039648, 0.84803486, 0.31782482,
       0.5152457 , 0.57930967, 0.07549394, 0.60946959, 0.32913898,
       0.66930308, 0.52355687, 0.44298197, 0.6010679 , 0.42368732,
       0.33375664, 0.42828484, 0.12141961, 0.61962503, 0.53549578,
       0.49110294, 0.37944444, 0.5865943 , 0.62343491, 0.82223686,
       0.06455837, 0.10214006, 0.79170627, 0.75369771, 0.74498663,
       0.6384874 , 0.22949371, 0.34069394, 0.71111011, 0.23282003,
       0.35394458, 0.60227066, 0.49807645, 0.74811863, 0.35021547,
       0.41467318, 0.66445777, 0.45493003, 0.27886276, 0.2742875 ,
  

 [(0, array([0.20367512, 0.25270237, 0.74711033, 0.67468157, 0.53569653,
       0.27046498, 0.35010265, 0.42388596, 0.53040324, 0.23093362,
       0.24603847, 0.60285946, 0.73081107, 0.36346333, 0.65211801,
       0.71832378, 0.66643732, 0.58560325, 0.59915617, 0.55007033,
       0.19568069, 0.37260778, 0.27130791, 0.62035233, 0.56618506,
       0.51817584, 0.58081112, 0.79039648, 0.84803486, 0.31782482,
       0.5152457 , 0.57930967, 0.07549394, 0.60946959, 0.32913898,
       0.66930308, 0.52355687, 0.44298197, 0.6010679 , 0.42368732,
       0.33375664, 0.42828484, 0.12141961, 0.61962503, 0.53549578,
       0.49110294, 0.37944444, 0.5865943 , 0.62343491, 0.82223686,
       0.06455837, 0.10214006, 0.79170627, 0.75369771, 0.74498663,
       0.6384874 , 0.22949371, 0.34069394, 0.71111011, 0.23282003,
       0.35394458, 0.60227066, 0.49807645, 0.74811863, 0.35021547,
       0.41467318, 0.66445777, 0.45493003, 0.27886276, 0.2742875 ,
       0.75369771, 0.36382616, 0.40696736, 0.09656808, 0

 [(0, array([0.20367512, 0.25270237, 0.74711033, 0.67468157, 0.53569653,
       0.27046498, 0.35010265, 0.42388596, 0.53040324, 0.23093362,
       0.24603847, 0.60285946, 0.73081107, 0.36346333, 0.65211801,
       0.71832378, 0.66643732, 0.58560325, 0.59915617, 0.55007033,
       0.19568069, 0.37260778, 0.27130791, 0.62035233, 0.56618506,
       0.51817584, 0.58081112, 0.79039648, 0.84803486, 0.31782482,
       0.5152457 , 0.57930967, 0.07549394, 0.60946959, 0.32913898,
       0.66930308, 0.52355687, 0.44298197, 0.6010679 , 0.42368732,
       0.33375664, 0.42828484, 0.12141961, 0.61962503, 0.53549578,
       0.49110294, 0.37944444, 0.5865943 , 0.62343491, 0.82223686,
       0.06455837, 0.10214006, 0.79170627, 0.75369771, 0.74498663,
       0.6384874 , 0.22949371, 0.34069394, 0.71111011, 0.23282003,
       0.35394458, 0.60227066, 0.49807645, 0.74811863, 0.35021547,
       0.41467318, 0.66445777, 0.45493003, 0.27886276, 0.2742875 ,
       0.75369771, 0.36382616, 0.40696736, 0.09656808, 0

 [(0, array([0.20367512, 0.25270237, 0.74711033, 0.67468157, 0.53569653,
       0.27046498, 0.35010265, 0.42388596, 0.53040324, 0.23093362,
       0.24603847, 0.60285946, 0.73081107, 0.36346333, 0.65211801,
       0.71832378, 0.66643732, 0.58560325, 0.59915617, 0.55007033,
       0.19568069, 0.37260778, 0.27130791, 0.62035233, 0.56618506,
       0.51817584, 0.58081112, 0.79039648, 0.84803486, 0.31782482,
       0.5152457 , 0.57930967, 0.07549394, 0.60946959, 0.32913898,
       0.66930308, 0.52355687, 0.44298197, 0.6010679 , 0.42368732,
       0.33375664, 0.42828484, 0.12141961, 0.61962503, 0.53549578,
       0.49110294, 0.37944444, 0.5865943 , 0.62343491, 0.82223686,
       0.06455837, 0.10214006, 0.79170627, 0.75369771, 0.74498663,
       0.6384874 , 0.22949371, 0.34069394, 0.71111011, 0.23282003,
       0.35394458, 0.60227066, 0.49807645, 0.74811863, 0.35021547,
       0.41467318, 0.66445777, 0.45493003, 0.27886276, 0.2742875 ,
       0.75369771, 0.36382616, 0.40696736, 0.09656808, 0

In [44]:
# get support vectors
kernel = ["poly", "rbf", "sigmoid","linear"]
for ker in kernel:
    svm_cf = SVC(
    C=1.0,
    kernel=ker,
    degree=3,
    gamma='scale',
    coef0=0.0,
    shrinking=True,
    probability=False,
    tol=0.001,
    cache_size=200,
    class_weight=None,
    verbose=False,
    max_iter=-1,
    decision_function_shape='ovo',
    break_ties=False,
    random_state=None)
    svm_cf.fit(X_train_scale,y_train)
    y_pred = svm_cf.predict(X_test_scale)
    print("No of Support Vectors corrsponding class of kernel",ker,"are : \n",list(zip(svm_cf.classes_,svm_cf.n_support_)),"\n\n")
    

No of Support Vectors corrsponding class of kernel poly are : 
 [(0, 49), (1, 37), (2, 40), (3, 28), (4, 50), (5, 36), (6, 26), (7, 28)] 


No of Support Vectors corrsponding class of kernel rbf are : 
 [(0, 92), (1, 76), (2, 68), (3, 67), (4, 74), (5, 64), (6, 70), (7, 67)] 


No of Support Vectors corrsponding class of kernel sigmoid are : 
 [(0, 104), (1, 84), (2, 106), (3, 94), (4, 97), (5, 78), (6, 93), (7, 95)] 


No of Support Vectors corrsponding class of kernel linear are : 
 [(0, 80), (1, 64), (2, 52), (3, 49), (4, 67), (5, 51), (6, 55), (7, 43)] 




### Lets use SVM using the cross validation

In [45]:
estimator = SVC(
    C=1.0,
    kernel='rbf',
    degree=3,
    gamma='scale',
    coef0=0.0,
    shrinking=True,
    probability=False,
    tol=0.001,
    cache_size=200,
    class_weight=None,
    verbose=False,
    max_iter=-1,
    decision_function_shape='ovr',
    break_ties=False,
    random_state=None)


svm_cv_score = cross_val_score(estimator=estimator,X=X_train_scale,y=y_train,cv=5)
print("Cross Validation Score: ",svm_cv_score)
print("Cross Validation Mean Score: ",svm_cv_score.mean())

Cross Validation Score:  [0.96688742 1.         0.99333333 0.99333333 0.98666667]
Cross Validation Mean Score:  0.9880441501103752


### Lets apply the Gridsearch with CV

In [46]:
estimator_gs = SVC(
    C=1.0,
    kernel='rbf',
    degree=3,
    gamma='scale',
    coef0=0.0,
    shrinking=True,
    probability=False,
    tol=0.001,
    cache_size=200,
    class_weight=None,
    verbose=False,
    max_iter=-1,
    decision_function_shape='ovr',
    break_ties=False,
    random_state=None)

parameter =  {'kernel':['linear', 'rbf', 'poly', 'sigmoid'],
              'C':[0.1, 1.0, 10.0, 100.0, 1000.0],
              'gamma':[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}

svm_gs_clf = GridSearchCV(estimator=estimator_gs,param_grid=parameter,cv=5,scoring="accuracy")
svm_gs_clf.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1.0, 10.0, 100.0, 1000.0],
                         'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             scoring='accuracy')

In [47]:
svm_gs_clf.best_params_

{'C': 100.0, 'gamma': 0.1, 'kernel': 'rbf'}

In [48]:
svm_gs_clf.best_score_

0.9986754966887418